In [ ]:
from datasets import load_from_disk, load_dataset

common_voice_train = load_dataset("common_voice", "de", split="train", cache_dir="D:\Master\wsl\data")
common_voice_validation = load_dataset("common_voice", "de", split="validation", cache_dir="D:\Master\wsl\data")
common_voice_test = load_dataset("common_voice", "de", split="test", cache_dir="D:\Master\wsl\data")

In [ ]:
common_voice_train = common_voice_train.remove_columns(["path","accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_validation = common_voice_validation.remove_columns(["path","accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["path","accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

# Preprocess Text 

In [ ]:
import re
chars_to_keep = '[^A-Za-zäüöß ]+'

def remove_special_characters_chris(batch):
    batch["sentence"] = re.sub(chars_to_keep, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters_chris)
common_voice_validation = common_voice_validation.map(remove_special_characters_chris)
common_voice_test = common_voice_test.map(remove_special_characters_chris)

In [ ]:
common_voice_validation.save_to_disk("E:/Master/data/val_text")
common_voice_train.save_to_disk("E:/Master/data/train_text")
common_voice_test.save_to_disk("E:/Master/data/test_text")

# Load Text Dataset

In [1]:
from datasets import load_from_disk, load_dataset

train_text_dataset = load_from_disk("E:/Master/data/0_text/train_text")
val_text_dataset = load_from_disk("E:/Master/data/0_text/val_text")
test_text_dataset = load_from_disk("E:/Master/data/0_text/test_text")

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda112]

In [ ]:
!python -m spacy download en_core_web_trf
!python -m spacy download de_dep_news_trf

In [ ]:
!pip install spacy[transformers,cuda112]

In [ ]:
!python -m spacy download de_core_news_sm

In [2]:
import spacy

spacy.prefer_gpu()

True

# Load Spacy

In [2]:
import spacy

#spacy.prefer_gpu()
nlp = spacy.load("de_dep_news_trf")
nlp.max_length = 17000000

In [3]:
train_complete_text = ""
val_complete_text = ""
test_complete_text = ""

In [4]:
print(train_text_dataset.shape[0])
print(val_text_dataset.shape[0])
print(test_text_dataset.shape[0])

246525
15588
15588


## Convert Datasets to String

In [5]:
from tqdm import tqdm

for i in tqdm(range(train_text_dataset.shape[0])):
    train_complete_text = train_complete_text + train_text_dataset[i]['sentence']

for i in tqdm(range(val_text_dataset.shape[0])):
    val_complete_text = val_complete_text + val_text_dataset[i]['sentence']

for i in tqdm(range(test_text_dataset.shape[0])):
    test_complete_text = test_complete_text + test_text_dataset[i]['sentence']

100%|█████████████████████████████████████████████████████████████████████████| 15588/15588 [00:00<00:00, 32679.25it/s]


In [6]:
print(len(train_complete_text))
print(len(val_complete_text))
print(len(test_complete_text))

16656802
997429
976676


# Process Datasets with Spacy

In [7]:
from datetime import datetime

now = datetime.now()
print("start process train @ ", now)

train_doc_trf = nlp(train_complete_text, disable = ['ner', 'parser'])

now = datetime.now()
print("End process train @ ", now)

start process train @  2021-05-19 11:37:49.702686


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 18128999424 bytes. Buy new RAM!

In [219]:
with open('train_doc_sm.pickle', 'wb') as handle:
    pickle.dump(train_doc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [203]:
from datetime import datetime

now = datetime.now()
print("start process val @ ", now)

val_doc = nlp(val_complete_text, disable = ['ner', 'parser'])

now = datetime.now()
print("End process val @ ", now)

start process val @  2021-05-18 19:37:43.615021
End process val @  2021-05-18 19:42:20.076160


In [212]:
import pickle

with open('val_doc_trf.pickle', 'wb') as handle:
    pickle.dump(val_doc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [214]:
with open('val_doc_trf.pickle', 'rb') as handle:
    val_doc_trf = pickle.load(handle)

False


In [8]:
now = datetime.now()
print("start process test @ ", now)

test_doc = nlp(test_complete_text, disable = ['ner', 'parser'])

now = datetime.now()
print("End process test @ ", now)

start process test @  2021-05-18 14:42:02.360915
End process test @  2021-05-18 14:42:08.361938


# Analyse train Data

In [83]:
# all tokens that arent stop words or punctuations
train_all = [token.text
         for token in train_doc
         if not token.is_punct]

# all tokens that arent stop words or punctuations
train_words = [token.text
         for token in train_doc
         if not token.is_stop and not token.is_punct]

# noun tokens that arent stop words or punctuations
train_nouns = [token.text
         for token in train_doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "NOUN")]

In [121]:
for word in train_words:
    if(word == "ausgebeutet"):
        print("found!")

found!
found!
found!
found!
found!
found!
found!
found!
found!
found!


In [90]:
from collections import Counter

print("train all: ", len(train_all))
train_all_freq = Counter(train_all)
print("train words dict: ", len(train_all_freq))

print("train words: ", len(train_words))
train_words_freq = Counter(train_words)
print("train words dict: ", len(train_words_freq))

print("train nouns: ", len(train_nouns))
train_nouns_freq = Counter(train_nouns)
print("train nouns dict: ", len(train_nouns_freq))

train all:  2350690
train words dict:  168237
train words:  1138469
train words dict:  167707
train nouns:  143335
train nouns dict:  26890


### Lemma nouns test:

In [91]:
# noun tokens that arent stop words or punctuations
train_nouns_lemma = [token.lemma_
         for token in train_doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "NOUN")]

In [95]:
print("Lemma Noun Words:", len(train_nouns_lemma))

train_nouns_lemma_freq = Counter(train_nouns_lemma)

print("Lemma Noun freq Words", len(train_nouns_lemma_freq))

Lemma Words: 143335
Lemma freq Words 26758


In [101]:
train_nouns_lemma_freq_keys = train_nouns_lemma_freq.keys()

n = 11

for i in range(1,n):
    
    rare_lemma_words = []
    for word in train_nouns_lemma_freq_keys:
        
        if train_nouns_lemma_freq[word] <= i:
            rare_lemma_words.append(word)
        
    print("Number of rare Lemma words " + str(i) + " : ", len(rare_lemma_words))

Number of rare Lemma words 1 :  17082
Number of rare Lemma words 2 :  20678
Number of rare Lemma words 3 :  22212
Number of rare Lemma words 4 :  23109
Number of rare Lemma words 5 :  23670
Number of rare Lemma words 6 :  24042
Number of rare Lemma words 7 :  24333
Number of rare Lemma words 8 :  24578
Number of rare Lemma words 9 :  24782
Number of rare Lemma words 10 :  24939


In [108]:
train_rare_lemma_words = []
for word in train_nouns_lemma_freq_keys:
    if train_nouns_lemma_freq[word] <= 1:
        train_rare_lemma_words.append(word)

print(train_rare_lemma_words[5:100])

['schiebers', 'sturmes', 'piscina', 'tankdeckels', 'fertigungsanlage', 'kopie', 'tauernfensters', 'staatsuniversität', 'modellautos', 'hufen', 'rennleitung', 'schutzumschlag', 'highschoolzeit', 'kontrabassisten', 'homiletik', 'heiliggeistordens', 'beifall', 'geschichtspolitik', 'instandsetzungstruppe', 'merowingern', 'austreiben', 'sesklokultur', 'ovale', 'trajektorie', 'umsatzbeteiligung', 'gesellschafterversammlung', 'wood', 'trialog', 'granada', 'bildpunkt', 'einsatzfall', 'zarismus', 'feudalismus', 'realschulempfehlung', 'boxsack', 'versuchsanordnung', 'christin', 'xte', 'matheprüfung', 'weihnachtsbaumdekoration', 'modelltheorie', 'böhme', 'unterkieferknochen', 'ormandy', 'cholerakonferenz', 'arbeitsbewertung', 'handymusik', 'komponist', 'audiodaten', 'barchfeld', 'antonio', 'adour', 'humorvoll', 'master', 'freileitung', 'gesundheitsschutzniveau', 'ausgabenprogramm', 'schiffsunglück', 'friedenszeiten', 'mennonitengemeinde', 'leitungsfunktion', 'tritt', 'sauna', 'stiftungsprofessur'

### lemma words test:

In [136]:
train_words_lemma = [token.lemma_
         for token in train_doc
         if (not token.is_stop and
             not token.is_punct)]

print("Lemma Words:", len(train_words_lemma))
train_words_lemma_freq = Counter(train_words_lemma)
print("Lemma freq Words", len(train_words_lemma_freq))

Lemma Words: 1138469
Lemma freq Words 151687


In [135]:
for word in train_words:
    if word == "rutschte":
        print("found!")

train_words_freq['rutschte']

found!
found!
found!


3

In [138]:
train_words_lemma_freq_keys = train_words_lemma_freq.keys()

n = 11

for i in range(1,n):
    
    train_rare_words = []
    for word in train_words_lemma_freq_keys:
        
        if train_words_lemma_freq[word] <= i:
            train_rare_words.append(word)
        
    print("Number of rare Lemma words " + str(i) + " : ", len(train_rare_words))

Number of rare Lemma words 1 :  84290
Number of rare Lemma words 2 :  108559
Number of rare Lemma words 3 :  119610
Number of rare Lemma words 4 :  125815
Number of rare Lemma words 5 :  129823
Number of rare Lemma words 6 :  132562
Number of rare Lemma words 7 :  134655
Number of rare Lemma words 8 :  136284
Number of rare Lemma words 9 :  137614
Number of rare Lemma words 10 :  138702


In [126]:
train_rare_words = []
for word in train_words_freq_keys:
    if train_words_freq[word] <= 1:
        train_rare_words.append(word)

print(train_rare_words[5:100])

['alfried', 'abrakadabra', 'abdrücken', 'jadebusen', 'rohstoffknappheit', 'privatkraftwagen', 'indoarischen', 'schiebers', 'kartenleser', 'juniorengrandprixfinale', 'platzanweiser', 'sturmes', 'abkauft', 'fehlerkorrekturen', 'franzoseneinfall', 'knollig', 'obie', 'maissorten', 'piscina', 'landtagswahlkreis', 'turatta', 'mitanni', 'lscheich', 'geldkoffer', 'stahlfelgen', 'rutschen', 'tankdeckels', 'gerst', 'bestreicht', 'brötchenhälfte', 'ausgebeuteten', 'ausbeuter', 'mörderisch', 'verhandlungssache', 'luftkraftstoffgemisch', 'fertigungsanlage', 'emile', 'gomer', 'nassauidenstein', 'logoireihe', 'hereinholen', 'dünenstinkmorchel', 'bellman', 'cereus', 'tauernbach', 'schiefers', 'tauernfensters', 'gerichtsakten', 'ostwestfalens', 'funston', 'bambergs', 'czapski', 'staatsuniversität', 'vallaster', 'gifsuryvette', 'tuning', 'knolls', 'nauroth', 'zeitungswissenschaft', 'dietenhofen', 'armenisch', 'kunstgewerbemuseums', 'militärwaffen', 'lgötzen', 'kinderdentist', 'hilfst', 'zeigegeste', 'ko

## train words 

In [139]:
train_words_freq_keys = train_words_freq.keys()

n = 11

for i in range(0,n):
    
    rare_words = []
    for word in train_words_freq_keys:
        
        if train_words_freq[word] <= i:
            rare_words.append(word)
        
    print("Number of rare words " + str(i) + " : ", len(rare_words))

Number of rare words 0 :  0
Number of rare words 1 :  91121
Number of rare words 2 :  118265
Number of rare words 3 :  130694
Number of rare words 4 :  137805
Number of rare words 5 :  142409
Number of rare words 6 :  145643
Number of rare words 7 :  148076
Number of rare words 8 :  149990
Number of rare words 9 :  151534
Number of rare words 10 :  152818


In [141]:
train_rare_words = []
for word in train_words_freq_keys:
    if train_words_freq[word] <= 1:
        train_rare_words.append(word)
        
print(len(train_rare_words))
print(train_rare_words[:100])

91121
['drannehmen', 'marktverständnis', 'heilbäder', 'bildbearbeitungsprogramme', 'muttersohnkonflikt', 'alfried', 'abrakadabra', 'abdrücken', 'jadebusen', 'rohstoffknappheit', 'privatkraftwagen', 'indoarischen', 'schiebers', 'kartenleser', 'juniorengrandprixfinale', 'platzanweiser', 'sturmes', 'abkauft', 'fehlerkorrekturen', 'franzoseneinfall', 'knollig', 'obie', 'maissorten', 'piscina', 'landtagswahlkreis', 'turatta', 'mitanni', 'lscheich', 'geldkoffer', 'stahlfelgen', 'rutschen', 'tankdeckels', 'gerst', 'bestreicht', 'brötchenhälfte', 'ausgebeuteten', 'ausbeuter', 'mörderisch', 'verhandlungssache', 'luftkraftstoffgemisch', 'fertigungsanlage', 'emile', 'gomer', 'nassauidenstein', 'logoireihe', 'hereinholen', 'dünenstinkmorchel', 'bellman', 'cereus', 'tauernbach', 'schiefers', 'tauernfensters', 'gerichtsakten', 'ostwestfalens', 'funston', 'bambergs', 'czapski', 'staatsuniversität', 'vallaster', 'gifsuryvette', 'tuning', 'knolls', 'nauroth', 'zeitungswissenschaft', 'dietenhofen', 'arm

In [174]:
train_rare_words_2 = []
for word in train_words_freq_keys:
    if train_words_freq[word] == 2:
        train_rare_words_2.append(word)
        
print(len(train_rare_words_2))
print(train_rare_words_2[:100])

27144
['romakindern', 'gemeinschaftsmitteln', 'erzwungener', 'qibla', 'minarette', 'wohngegenden', 'düsseldorfs', 'feldzeichen', 'humperdincks', 'feldherren', 'saratoga', 'startrekgeschichte', 'ziggy', 'domgemeinde', 'cell', 'verriss', 'reichranicki', 'hormonhaushalt', 'ogden', 'empfohlene', 'dustin', 'janka', 'unermesslich', 'kyrillischen', 'leberwurst', 'einzahlt', 'verschlingen', 'treffe', 'kettenhemd', 'zündkerze', 'aerts', 'radrennfahrers', 'modulare', 'kimbern', 'atomubooten', 'konfliktfall', 'bermel', 'convergence', 'triskele', 'badearzt', 'separatismus', 'abschlüssen', 'absurdes', 'frischluft', 'mittelasien', 'subg', 'großgruppe', 'schutzbereich', 'belvedere', 'ungedruckt', 'badoglio', 'harare', 'modellautos', 'selbstgespräche', 'oberwürzbach', 'niederwürzbach', 'kunstlehrer', 'südwestrundfunk', 'gemischtes', 'stereozentrum', 'plutos', 'rillen', 'promis', 'entwarnung', 'teilprobleme', 'ahoi', 'landratten', 'halbton', 'winkelmann', 'flecke', 'velum', 'brill', 'schattendasein', '

In [182]:
train_rare_words_3 = []
for word in train_words_freq_keys:
    if train_words_freq[word] == 3:
        train_rare_words_3.append(word)
        
print(len(train_rare_words_3))
print(train_rare_words_3[:100])

12429
['wüstenbildung', 'totales', 'markanter', 'polarstern', 'kuppeln', 'landesmuseums', 'steinbock', 'moyne', 'leitthema', 'wechselhaft', 'kater', 'itzehoe', 'kriegsteilnehmer', 'artilleriebrigade', 'falschgeld', 'dry', 'analytischer', 'karasek', 'tabs', 'nässe', 'reifendruck', 'sachse', 'imperiums', 'tabea', 'keltisches', 'schmiedeberg', 'verursache', 'rechtwinkligen', 'untergattung', 'judenburg', 'swahili', 'grundstoff', 'straßenverkehrsordnung', 'perfektes', 'lillehammer', 'kompost', 'hacke', 'chiraler', 'quarterback', 'alkoholproblem', 'synthetisieren', 'echternach', 'tampico', 'unterernährung', 'portiert', 'militärgericht', 'polemik', 'aphorismen', 'synchronisieren', 'truppenkommando', 'gleitet', 'doppelphaeton', 'subregens', 'herold', 'taian', 'schollen', 'erinnerungskultur', 'lauenburg', 'wurfarm', 'verbal', 'dankend', 'nervenzusammenbruch', 'quintus', 'taktiker', 'umweltpolitische', 'kernreaktor', 'geometrien', 'arbeitsgang', 'eskalierte', 'maschinengewehren', 'sachverständig

In [184]:
train_rare_words_4 = []
for word in train_words_freq_keys:
    if train_words_freq[word] == 4:
        train_rare_words_4.append(word)
        
print(len(train_rare_words_4))
print(train_rare_words_4[:100])

7111
['haushaltslage', 'konfliktparteien', 'oberflächenwasser', 'teuersten', 'fliegende', 'rückten', 'gulasch', 'marmelade', 'fraktionslos', 'mittelalterlich', 'friedliebenden', 'irvine', 'personalunion', 'biomüll', 'entsorgen', 'safran', 'gel', 'weserbergland', 'ex', 'schwindelfreiheit', 'reichtums', 'hauptroute', 'königswinter', 'akute', 'gegliederte', 'kurie', 'zungenblüten', 'bakterienstamm', 'unverwechselbaren', 'leck', 'wertebereich', 'sehenswerten', 'mirror', 'kologischen', 'augusto', 'knolle', 'leistungsniveau', 'coda', 'augenlider', 'ovale', 'gabriela', 'gepanzerte', 'innenpolitik', 'bronzerang', 'sarkophage', 'bestplatzierten', 'tillmann', 'baugewerbe', 'auszunutzen', 'bischofssitzes', 'verlustreichen', 'mons', 'unbefestigte', 'pritschenwagen', 'ausgewiesenen', 'erließ', 'ratifizierungsprozess', 'frust', 'pastoren', 'unternehmensgeschichte', 'gesetzmäßigkeit', 'gefäßen', 'urteils', 'christin', 'kegelförmige', 'flutwelle', 'handelsregister', 'resultierten', 'parodien', 'ladisl

In [185]:
train_rare_words_5 = []
for word in train_words_freq_keys:
    if train_words_freq[word] == 5:
        train_rare_words_5.append(word)
        
print(len(train_rare_words_5))
print(train_rare_words_5[:100])

4604
['besänftigen', 'erskine', 'unbeeinflusst', 'sejm', 'sprachgruppe', 'resultaten', 'geschäftspartner', 'thing', 'feindschaft', 'aromatische', 'platzgründen', 'feigen', 'erzieherin', 'statements', 'autoritäre', 'arendt', 'gunther', 'markthalle', 'kristallinen', 'infizierte', 'einzelausstellungen', 'überstimmt', 'trockenes', 'oberschenkel', 'betonten', 'jahrgangsstufe', 'mangas', 'highschoolzeit', 'lam', 'anfallen', 'brass', 'erler', 'kosaken', 'menzel', 'kriegsgräberfürsorge', 'barbaro', 'bindestrich', 'stiefel', 'wiedereintritt', 'austrieb', 'stängels', 'fidschi', 'galloway', 'eisenbahnverbindungen', 'aprikosen', 'biosynthese', 'dargebracht', 'einstweiligen', 'neugründungen', 'gräberfelder', 'volksversammlung', 'urbane', 'unverkennbar', 'brikettfabrik', 'kavallerieregiment', 'funkhaus', 'eg', 'amtlich', 'anteilen', 'flag', 'tierversuchen', 'ausschliesslich', 'mächtiger', 'unvollständigen', 'beigefügt', 'klagt', 'socken', 'stadtpfarrer', 'gemeinsamkeit', 'landesverwaltung', 'durchga

In [195]:
train_rare_words_10 = []
for word in train_words_freq_keys:
    if train_words_freq[word] == 10:
        train_rare_words_10.append(word)
        
print(len(train_rare_words_10))
print(train_rare_words_10[:100])

1284
['bertha', 'dänischer', 'verdankte', 'grass', 'herrschten', 'informationsgesellschaft', 'hallen', 'schleusen', 'hinterlegt', 'praktikabel', 'soliden', 'handwerks', 'bremerhaven', 'kirchenschiffs', 'verkehrswege', 'überwiegenden', 'gebäudekomplex', 'karosserieform', 'bezirksfreien', 'feststellbar', 'vertikal', 'begleiteten', 'getränk', 'terminen', 'unternehmungen', 'zusatzstoffe', 'klimatischen', 'möller', 'eisenerz', 'musiktheater', 'auktionen', 'wood', 'vitorino', 'süditalien', 'sauerland', 'griffin', 'überlieferten', 'totalen', 'mose', 'schleswigholsteins', 'bishop', 'zufahrt', 'zögerlich', 'hendrix', 'findest', 'einschränken', 'hauptinsel', 'floss', 'schulischen', 'fairness', 'dubai', 'inhalts', 'quatsch', 'größenordnung', 'industriekultur', 'ausreden', 'veronika', 'festgesetzt', 'verzweiflung', 'falsches', 'bernardo', 'therapien', 'beklagt', 'altäre', 'ostfriesland', 'überlappen', 'seitental', 'bankkaufmann', 'züchter', 'flüchteten', 'bringe', 'klausel', 'thüringischen', 'pass

## Stem experiments

In [142]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("german")

train_rare_stem_words = []

for word in train_rare_words:
    train_rare_stem_words.append(stemmer.stem(word))

print("Rare Stem Words:", len(train_rare_stem_words))

train_rare_stem_words_freq = Counter(train_rare_stem_words)

print("Rare stem Words freq", len(train_rare_stem_words_freq))

Rare Stem Words: 91121
Rare stem Words freq 83273


In [148]:
train_rare_stem_words_2 = []

train_rare_stem_words_freq_keys = train_rare_stem_words_freq.keys()

for word in train_rare_stem_words_freq:
    if train_rare_stem_words_freq[word] <= 1:
        train_rare_stem_words_2.append(word)
        
print(train_rare_stem_words_2[:100])

['drannehm', 'marktverstandnis', 'heilbad', 'muttersohnkonflikt', 'alfried', 'abrakadabra', 'jadebus', 'rohstoffknapp', 'privatkraftwag', 'indoar', 'schieb', 'kartenles', 'juniorengrandprixfinal', 'platzanweis', 'abkauft', 'fehlerkorrektur', 'franzoseneinfall', 'knollig', 'obi', 'maissort', 'piscina', 'landtagswahlkreis', 'turatta', 'mitanni', 'lscheich', 'geldkoff', 'stahlfelg', 'tankdeckel', 'gerst', 'bestreicht', 'brotchenhalft', 'ausgebeutet', 'ausbeut', 'morder', 'verhandlungssach', 'luftkraftstoffgem', 'emil', 'gom', 'nassauidenstein', 'logoireih', 'hereinhol', 'dunenstinkmorchel', 'bellman', 'cereus', 'tauernbach', 'tauernfenst', 'gerichtsakt', 'funston', 'bamberg', 'czapski', 'staatsuniversitat', 'vallast', 'gifsuryvett', 'tuning', 'knoll', 'nauroth', 'dietenhof', 'armen', 'kunstgewerbemuseum', 'militarwaff', 'lgotz', 'kinderdentist', 'zeigeg', 'korrekt', 'dervi', 'erolu', 'monogam', 'vogl', 'handelsgeschaft', 'bohlau', 'quellenfrei', 'gnesiolutheran', 'philippist', 'zugerechne

In [48]:
print(len(train_word_stem_freq))

rare_stem_words = []

train_word_stem_freq_keys = train_word_stem_freq.keys()

for word in train_word_stem_freq:
    if train_word_stem_freq[word] < 2:
        rare_stem_words.append(word)
        
print(len(rare_stem_words))

123944
66504


In [51]:
train_word_stem_lemma_freq = Counter(train_words_stemmed_lemma_List)

print(len(train_word_stem_lemma_freq))

rare_stem_lemma_words = []

train_word_stem_freq_keys = train_word_stem_lemma_freq.keys()

for word in train_word_stem_lemma_freq:
    if train_word_stem_lemma_freq[word] < 2:
        rare_stem_lemma_words.append(word)
        
print(len(rare_stem_lemma_words))

120003
65216


# Analyse Val Data

In [204]:
val_words = [token.text
         for token in val_doc
         if not token.is_stop and not token.is_punct]

In [215]:
val_words_trf = [token.text
         for token in val_doc_trf
         if not token.is_stop and not token.is_punct]

In [216]:
print(val_words == val_words_trf)

True


In [207]:
val_lemma = [token.lemma_
         for token in val_doc
         if not token.is_stop and not token.is_punct]

In [208]:
print("Val lemma: ", len(val_lemma))
val_lemma_freq = Counter(val_lemma)
print("Val words dict: ", len(val_lemma_freq))

Val lemma:  68675
Val words dict:  26350


In [210]:
rare_val_lemma = []

val_lemma_freq_keys = val_lemma_freq.keys()

for word in val_lemma_freq_keys:
    if val_lemma_freq[word] <= 1:
        rare_val_lemma.append(word)

print(len(rare_val_lemma))
print(rare_val_lemma[:100])

18099
['feick', 'geflügel', 'schweinezucht', 'beschäftigungen', 'okci', 'fischingen', 'westsahara', 'kisten', 'protein', 'lebenswichtig', 'kostüm', 'hingucker', 'frühchen', 'postkarte', 'christiane', 'matheprüfung', 'mithelfen', 'maisfeldes', 'labyrinth', 'aufzubocken', 'tschüss', 'streunend', 'kater', 'massenweise', 'spatzen', 'frostschutzmittel', 'schnupfen', 'krukenbergspindel', 'saintdenis', 'kaspar', 'wolfersweiler', 'unfalls', 'schlackerst', 'merlin', 'fleisch', 'fleischwolf', 'aufgeditscht', 'haushaltsbeschränkung', 'kelsos', 'harfe', 'auffassungen', 'rauschgifthandels', 'stabhochsprung', 'enttäuschung', 'doppelkabine', 'burmanniaarten', 'daddeln', 'schwielen', 'headliner', 'saarlouis', 'abschnittweise', 'apotheker', 'geschlechtsdimorphismus', 'frühstückt', 'personalleiter', 'ausnahmetalent', 'unzüchtiges', 'krokodilklemmen', 'ankurbeln', 'ladenbesitzer', 'schiebetür', 'zahnlücke', 'ausflippen', 'erwidern', 'olga', 'ernten', 'säen', 'umsiedeln', 'fix', 'konstante', 'landbesitz',

In [205]:
print("Val words: ", len(val_words))
val_words_freq = Counter(val_words)
print("Val words dict: ", len(val_words_freq))

Val words:  68675
Val words dict:  29735


In [211]:
rare_val_words = []

val_words_freq_keys = val_words_freq.keys()

for word in val_words_freq_keys:
    if val_words_freq[word] <= 1:
        rare_val_words.append(word)

print(len(rare_val_words))
print(rare_val_words[:100])

20551
['feick', 'geflügel', 'schweinezucht', 'beschäftigungen', 'okci', 'fischingen', 'westsahara', 'kisten', 'stopfen', 'protein', 'lebenswichtig', 'kostüm', 'hingucker', 'frühchen', 'errechneten', 'postkarte', 'christiane', 'matheprüfung', 'mitgeholfen', 'maisfeldes', 'labyrinth', 'aufzubocken', 'tschüss', 'streunender', 'kater', 'massenweise', 'spatzen', 'natürlichem', 'frostschutzmittel', 'harmloser', 'schnupfen', 'krukenbergspindel', 'saintdenis', 'kaspar', 'wolfersweiler', 'unfalls', 'abfällt', 'schlackerst', 'merlin', 'fleisch', 'fleischwolf', 'versunken', 'aufgeditscht', 'haushaltsbeschränkung', 'einzubinden', 'kelsos', 'fremde', 'harfe', 'auffassungen', 'rauschgifthandels', 'stabhochsprung', 'enttäuschung', 'doppelkabine', 'burmanniaarten', 'daddeln', 'schwielen', 'headliner', 'saarlouis', 'abschnittweise', 'apotheker', 'ausgeprägter', 'geschlechtsdimorphismus', 'frühstückt', 'wundern', 'personalleiter', 'ausnahmetalent', 'unzüchtiges', 'krokodilklemmen', 'angekurbelt', 'laden

In [206]:
val_train_set = []
for word in train_rare_words:
    if val_words_freq[word] >= 1:
        val_train_set.append(word)
        
print(len(val_train_set))

3232


# Analyse Test Data

In [156]:
test_words = [token.text
         for token in test_doc
         if not token.is_stop and not token.is_punct]

In [193]:
test_all = [token.text
         for token in test_doc
         if not token.is_punct]

print("Test all: ", len(test_all))
test_all_freq = Counter(test_all)
print("Test all dict: ", len(test_all_freq))

Test all:  140713
Test all dict:  30042


In [157]:
print("Test words: ", len(test_words))
test_words_freq = Counter(test_words)
print("Test words dict: ", len(test_words_freq))

Test words:  67545
Test words dict:  29553


In [178]:
test_train_set = []
test_train_count = 0
for word in train_rare_words:
    if test_words_freq[word] >= 1:
        test_train_count = test_train_count + test_words_freq[word]
        test_train_set.append(word)
        
print(len(test_train_set))
print(test_train_count)

3235
3418


In [179]:
test_train_set_2 = []
test_train_count_2 = 0
for word in train_rare_words_2:
    if test_words_freq[word] >= 1:
        test_train_count_2 = test_train_count_2 + test_words_freq[word]
        test_train_set_2.append(word)
        
print(len(test_train_set_2))
print(test_train_count_2)

2239
2446


In [181]:
test_train_set_3 = []
test_train_count_3 = 0
for word in train_rare_words_3:
    if test_words_freq[word] >= 1:
        test_train_count_3 = test_train_count_3 + test_words_freq[word]
        test_train_set_3.append(word)
        
print(len(test_train_set_3))
print(test_train_count_3)

1538
1758


In [200]:
test_train_set_4 = []
test_train_count_4 = 0
for word in train_rare_words_4:
    if test_words_freq[word] >= 1:
        test_train_count_4 = test_train_count_4 + test_words_freq[word]
        test_train_set_4.append(word)
    if test_words_freq[word] >= 4:
        print(word)
        
print(len(test_train_set_4))
print(test_train_count_4)

drüben
eingezeichnet
packung
1280
1486


In [187]:
test_train_set_5 = []
test_train_count_5 = 0
for word in train_rare_words_5:
    if test_words_freq[word] >= 1:
        test_train_count_5 = test_train_count_5 + test_words_freq[word]
        test_train_set_5.append(word)
        
print(len(test_train_set_5))
print(test_train_count_5)

1063
1271


In [201]:
test_train_set_10 = []
test_train_count_10 = 0
for word in train_rare_words_10:
    if test_words_freq[word] >= 1:
        test_train_count_10 = test_train_count_10 + test_words_freq[word]
        test_train_set_10.append(word)
    if test_words_freq[word] >= 4:
        print(word)
        
print(len(test_train_set_10))
print(test_train_count_10)

bremerhaven
sahne
enzyme
geste
regierungssitz
bewaffnet
teppich
erwachsen
versteckte
segen
isst
erzählte
verbraucht
pole
warmen
angetroffen
549
808


# Check WER

In [159]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-xlsr-ger-chris/checkpoint-51000")

In [160]:
def map_to_result(batch):
    
    model.to("cuda")
    input_values = processor(
          batch["speech"], 
          sampling_rate=batch["sampling_rate"], 
          return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

In [161]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [162]:
from datasets import load_from_disk, load_dataset

test_sampled = load_from_disk("E:/Master/data/test_sampled")

In [164]:
results = test_sampled.map(map_to_result)

  0%|          | 0/15588 [00:00<?, ?ex/s]

In [165]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.147


In [167]:
results.save_to_disk("results51_test")

In [169]:
result_complete_text = ""
for i in tqdm(range(results.shape[0])):
    result_complete_text = result_complete_text + results[i]['pred_str']

100%|████████████████████████████████████████████████████████████████████████████| 15588/15588 [06:43<00:00, 38.67it/s]


In [170]:
result_doc = nlp(result_complete_text, disable = ['ner', 'parser'])

In [220]:
result_doc_trf = nlp(result_complete_text, disable = ['ner', 'parser'])

In [221]:
with open('result_doc_trf.pickle', 'wb') as handle:
    pickle.dump(result_doc_trf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [171]:
result_words = [token.text
         for token in result_doc
         if not token.is_stop and not token.is_punct]

print("Result words: ", len(result_words))
result_words_freq = Counter(result_words)
print("Result words dict: ", len(result_words_freq))

Result words:  66319
Result words dict:  40662


In [194]:
result_all = [token.text
         for token in result_doc
         if not token.is_punct]

print("Result words: ", len(result_all))
result_all_freq = Counter(result_all)
print("Result words dict: ", len(result_all_freq))

Result words:  123501
Result words dict:  41131


In [190]:
print("Test words dict: ", len(test_words_freq))

test_words_freq_keys = test_words_freq.keys()

i = 0
for word in test_words_freq_keys:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

Test words dict:  29553
17017


In [172]:
i=0
for word in test_train_set:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

1348


In [176]:
i=0
for word in test_train_set_2:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

1058


In [183]:
i=0
for word in test_train_set_3:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

740


In [188]:
i=0
for word in test_train_set_4:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

616


In [189]:
i=0
for word in test_train_set_5:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

543


In [197]:
i=0
for word in test_train_set_10:
    if result_words_freq[word] >= 1:
        i = i+1
print(i)

348
